In [145]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
import pandas as pd
import xgboost
import os
import pickle

In [146]:
df_2015 = pd.read_csv("../2015_baby/data/master.csv")
df_2016 = pd.read_csv("../2016_baby/data/master.csv")
df_2017 = pd.read_csv("../2017_baby/data/master.csv")
df_2018 = pd.read_csv("../2018_baby/data/master.csv")
df_2014 = pd.read_csv("../2014_baby/data/master.csv")

In [147]:
df_master = df_2015.append(df_2016)
df_master = df_master.append(df_2017)
df_master = df_master.append(df_2018)
df_master = df_master.append(df_2014)
df_master = df_master.reset_index()
df_master = df_master.drop(["index"], axis=1)

In [148]:
non_stats_columns = ["name_x", "date", "name_y", "opp_name_x"]
stats_columns = ["assists_game_x", "blocks_game_x", "points_game_x", "total_rebounds_game_x", "assists_game_y", "blocks_game_y", "points_game_y", "total_rebounds_game_y", "outcomes_game_x"]

df_test = df_master.drop(non_stats_columns, axis=1)
df_test

,status_x,assists_game_x,assists_avg_x,att_field_goals_avg_x,att_free_throws_avg_x,att_three_points_avg_x,blocks_game_x,blocks_avg_x,defensive_rebounds_avg_x,made_field_goals_avg_x,...,opp_made_field_goals_10MA_y,opp_made_free_throws_10MA_y,opp_made_three_point_10MA_y,opp_offensive_rebounds_10MA_y,opp_personal_fouls_10MA_y,opp_points_10MA_y,opp_steals_10MA_y,opp_turnovers_10MA_y,total_rebounds_10MA_y,opp_total_rebounds_10MA_y
0,0,22.0,26.666667,82.916667,19.416667,24.916667,7.0,4.916667,31.833333,39.083333,...,38.6,16.9,9.5,9.6,18.6,103.6,9.0,13.8,45.0,43.3
1,1,23.0,26.307692,82.384615,19.000000,25.230769,6.0,5.076923,32.153846,38.615385,...,40.3,17.7,9.3,10.2,21.2,107.6,9.2,17.1,44.1,43.5
2,0,22.0,26.071429,82.857143,19.357143,25.571429,4.0,5.142857,32.214286,38.571429,...,37.0,17.5,7.2,9.8,22.9,98.7,7.5,12.7,45.4,40.4
3,1,33.0,25.800000,83.066667,19.733333,25.733333,7.0,5.066667,31.800000,38.133333,...,35.1,19.5,7.1,11.4,20.2,96.8,9.0,17.9,44.8,42.7
4,0,14.0,26.250000,82.875000,19.875000,25.625000,7.0,5.187500,32.125000,38.562500,...,39.3,15.5,9.9,8.9,21.1,104.0,8.6,15.2,45.7,40.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11551,0,21.0,24.045977,82.862069,21.770115,17.275862,1.0,4.689655,34.356322,38.333333,...,36.8,16.8,8.9,10.2,18.3,99.3,7.5,14.7,44.3,46.1
11552,1,27.0,24.011364,82.852273,21.681818,17.329545,5.0,4.647727,34.340909,38.295455,...,36.4,15.5,9.2,9.5,18.5,97.5,7.6,14.4,44.9,45.0
11553,1,21.0,24.044944,82.797753,21.764045,17.449438,8.0,4.651685,34.337079,38.280899,...,35.8,16.1,8.9,9.5,19.0,96.6,7.4,14.0,45.0,45.4
11554,0,20.0,24.011111,82.822222,21.711111,17.544444,5.0,4.688889,34.300000,38.277778,...,36.5,15.2,9.4,9.8,19.7,97.6,6.8,13.5,44.5,44.5


In [149]:
for column in df_test:
    splits = column.split("_")
    if "game" in splits:
        print(column)

assists_game_x
blocks_game_x
points_game_x
outcomes_game_x
total_rebounds_game_x
assists_game_y
blocks_game_y
points_game_y
total_rebounds_game_y


In [150]:
x_train = df_test.drop(stats_columns, axis=1)
stats_columns = ["outcomes_game_x"]
for column in stats_columns:
    # Step 1
    params = {
        "n_estimators": [400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]
    }
    classifier = xgboost.XGBRegressor()
    
    y_train = df_test[column]
    random_search = RandomizedSearchCV(classifier,param_distributions=params,scoring="neg_mean_absolute_error",n_iter=13,n_jobs=-1,cv=5,verbose=3)
    random_search.fit(x_train, y_train)
    best = random_search.best_params_

    classifier = xgboost.XGBRegressor(n_estimators=best["n_estimators"])

    # step 2
    params = {
        "max_depth": [3, 4, 5, 6, 7, 9, 10, 12]
    }

    random_search = RandomizedSearchCV(classifier,param_distributions=params,scoring="neg_mean_absolute_error",n_iter=8,n_jobs=-1,cv=5,verbose=3)
    random_search.fit(x_train, y_train)
    best_test = random_search.best_params_

    best["max_depth"] = best_test["max_depth"]
    classifier = xgboost.XGBRegressor(n_estimators=best["n_estimators"], max_depth = best["max_depth"])
    
    
    # Step 3
    params = {
        "learning_rate": [0.01, 0.03, 0.05, 0.07, 0.09, 0.1],
        "subsample": [0.8, 0.85, 0.9, 0.95, 1],
        "gamma": [0, 1, 5,],
        "colsample_bytree": [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    }

    random_search = RandomizedSearchCV(classifier,param_distributions=params,scoring="neg_mean_absolute_error",n_iter=50,n_jobs=-1,cv=5,verbose=3)
    random_search.fit(x_train, y_train)
    best_test = random_search.best_params_

    best["learning_rate"] = best_test["learning_rate"]
    best["subsample"] = best_test["subsample"]
    best["gamma"] = best_test["gamma"]
    best["colsample_bytree"] = best_test["colsample_bytree"]

    classifier = xgboost.XGBRegressor(n_estimators=best["n_estimators"], max_depth = best["max_depth"], learning_rate = best["learning_rate"], subsample=best["subsample"], gamma=best["gamma"], colsample_bytree=best["colsample_bytree"])
    
    classifier.fit(x_train,y_train)

    with open('model_pickle_' + column + '.pickle', 'wb') as f:
        pickle.dump(classifier, f)

Fitting 5 folds for each of 13 candidates, totalling 65 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed: 22.7min finished


[23:53:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 17.9min finished


[00:12:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 33.3min finished


[00:46:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:46:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
